# Freeze-in CFD Demo

This notebook demonstrates the **ODE benchmark** and the **CFD-style finite-volume PDE** for the freeze-in yield.

We work with the dimensionless variable $x \equiv \frac{m_\chi}{T}$ and define the comoving yield $Y \equiv \frac{n_\chi}{s}$.

For decays $A\to\chi\chi$, the reference ODE is
$\frac{dY}{dx} \;=\; \frac{1}{s(T)}\;\frac{C(T)}{H(T)\,x},\qquad C(T)=2\,\Gamma_A\,n_A^{\rm eq}(T).$

The momentum–space PDE for $f_\chi(p,x)$ is
$\partial_x f_\chi \;+\; \partial_p\!\Big(\frac{-p}{x}\,f_\chi\Big) \;=\; S(x,p),$
with the **source normalization**
$\int_0^\infty p^2 \, S(x,p)\,dp \;=\; \frac{2\pi^2}{g_\chi}\,\frac{C(T)}{H(T)\,x},$
so that the **source-only yield** matches the ODE benchmark.


## Set up

Add local `src/` to `sys.path`.

In [ ]:

import os, sys, numpy as np, matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

ROOT = os.path.abspath(os.path.join(os.getcwd()))
SRC = os.path.join(ROOT, "src")
if SRC not in sys.path:
    sys.path.insert(0, SRC)

from freezein_fv.ivp_reference import solve_reference
from freezein_fv.pde_solver import march_pde


## Yield comparison (ODE vs PDE)

Vertical line marks the ODE→PDE switch.

In [ ]:

ref = solve_reference()
x_ref, Y_ref = ref['x_plot'], ref['Y']

out = march_pde(Np=600, Nx=2500, inj_width=0.02, scheme='muscl',
                time_integrator='rk2', x_switch=1e-2)
x_pde = out['x']
Y_src = out['Y_src_hist']
Y_fromf = out['Y_hist']

plt.figure(figsize=(8.4,4.8))
plt.loglog(x_ref, Y_ref, label='ODE (solve_ivp)')
plt.loglog(x_pde, Y_src, '--', label='PDE (source-only)')
plt.loglog(x_pde, Y_fromf, ':', label='PDE (integrated f)')
plt.axvline(1e-2, ls='--', lw=1)
plt.text(1.05e-2, Y_ref[np.searchsorted(x_ref,1e-2)], 'ODE → PDE', fontsize=9)
plt.xlabel(r'$x = m_\chi/T$'); plt.ylabel(r'$Y = n_\chi/s$'); plt.title('Yield comparison')
plt.grid(True, which='both', ls=':'); plt.legend(); plt.tight_layout()
plt.show()

print('Final Y (ODE)         :', Y_ref[-1])
print('Final Y (PDE src-only):', Y_src[-1])
print('Final Y (integrated f):', Y_fromf[-1])


## Momentum-space snapshots of $f_\chi(p)$

In [ ]:

from freezein_fv.physics import mA
snaps = [1e-2, 3e-2, 1e-1, 3e-1, 1.0]
out2 = march_pde(Np=600, Nx=2500, inj_width=0.02, scheme='muscl', time_integrator='rk2',
                 x_switch=1e-2, snapshots_x=snaps)
S = out2['snapshots']
items = sorted(S.items(), key=lambda kv: kv[1]['x'])

# Absolute p^2 f
plt.figure(figsize=(8.4,4.8))
for req_x, data in items:
    p, f, x_act = data['p'], data['f'], data['x']
    plt.loglog(p, p**2*f, label=f'x≈{x_act:.2e} (req {req_x:.0e})')
plt.axvline(0.5*mA, ls='--', lw=1)
plt.xlabel('p [GeV]'); plt.ylabel(r'$p^2 f_\chi(p)$ (arb.)')
plt.title('Momentum snapshots: number integrand'); plt.grid(True, which='both', ls=':'); plt.legend(); plt.tight_layout()
plt.show()

# Normalized shape
plt.figure(figsize=(8.4,4.8))
for req_x, data in items:
    p, f, x_act = data['p'], data['f'], data['x']
    area = np.trapz(p**2*f, p)
    plt.loglog(p, (p**2*f)/(area+1e-300), label=f'x≈{x_act:.2e}')
plt.axvline(0.5*mA, ls='--', lw=1)
plt.xlabel('p [GeV]'); plt.ylabel(r'$\frac{p^2 f_\chi(p)}{\int p^2 f_\chi dp}$')
plt.title('Momentum snapshots: shapes (normalized)'); plt.grid(True, which='both', ls=':'); plt.legend(); plt.tight_layout()
plt.show()

# Energy integrand p^3 f
plt.figure(figsize=(8.4,4.8))
for req_x, data in items:
    p, f, x_act = data['p'], data['f'], data['x']
    plt.loglog(p, p**3*f, label=f'x≈{x_act:.2e}')
plt.axvline(0.5*mA, ls='--', lw=1)
plt.xlabel('p [GeV]'); plt.ylabel(r'$p^3 f_\chi(p)$ (arb.)')
plt.title('Momentum snapshots: energy integrand'); plt.grid(True, which='both', ls=':'); plt.legend(); plt.tight_layout()
plt.show()
